# **Proyecto Integrador - Avance 5. Modelo Final**
## **Tecnologico de Monterrey**
------------------------------------------------------------------
### Profa. Dra. Grettel Barceló Alonso

### Prof. Dr. Luis Eduardo Falcón Morales

### Profa. Verónica Sandra Guzmán de Valle
------------------------------------------------------------------
### Marcela Alejandra Rosales Jiménez - A01032022
### José Antonio Mendoza Castro - A01794067

## **Resumen**

#### **En el presente Notebook se busca analizar la base de datos *Woven Traffic Safety* (WTS) la cual fue usada en *The 8th AI City Challange* realizado en 2024, el objetivo es poder presentar herramientas de Inteligencia Artificial que permitan disminuir conflictos viales. La base consta de videos y transcripciones de los mismos en los que se muestran eventos actuados entre peatones, ciclistas y automoviles. Para el analisis se obtuvierion nuevas transcripciones de los videos utilizando Gemini en un Notebook de Colab Enterprise de Google Cloud Platform (GCP) para una muestra aleatoria de 150 videos. Se utilizaron los siguientes prompts para la generacion de las nuevas transcripciones:**

*   #### **Look through each frame in the video carefully and answer the question.**
*   #### **Only base your answers strictly on what information is available in the video attached.**
*   #### **Do not make up any information that is not part of the video and do not be too verbose.**
*   #### **Questions:**
*   #### **Give me a detail description of the video, related to the safety of pedestrians with respect to vehicles.**
*   #### **In the video, is there a collision or accident (contact or hit between pedestrians and vehicles) for pedestrians with respect to vehicles?**

#### **Una vez listos los nuevos textos se prosiguio con el analisis en el se prueba:**


*   #### **Regresion Logistica**
*   #### **Red Neuronal**
*   #### **fastText**
*   #### **Zero-shot con la libreria Transformers**

#### **Para evaluar la clasificacion se utilizan las metricas Accuracy y Recall. Finalmente se llega a que la clasificacion obtenida con el Zero-shot tiene la mejor ejecucion y que podria ser explorada con mayor profundidad en analisis futuros.**





### **Preparacion del ambiente**

In [ ]:
!pip install --upgrade google-cloud-aiplatform

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 55.7 MB/s eta 0:00:00
  Attempting uninstall: google-cloud-aiplatform
    Found existing installation: google-cloud-aiplatform 1.70.0
    Uninstalling google-cloud-aiplatform-1.70.0:
      Successfully uninstalled google-cloud-aiplatform-1.70.0


In [ ]:
!pip install fasttext

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached pybind11-2.13.6-py3-none-any.whl.metadata (9.5 kB)
Using cached pybind11-2.13.6-py3-none-any.whl (243 kB)
  Created wheel for fasttext: filename=fasttext-0.9.3-cp310-cp310-linux_x86_64.whl size=4296187 sha256=0aed579ecdf8c045876967b51e20e1602730dea7a6892f4483d6c2de4f322fb4
  Stored in directory: /root/.cache/pip/wheels/0d/a2/00/81db54d3e6a8199b829d58e02cec2ddb20ce3e59fad8d3c92a
Successfully built fasttext


In [ ]:
!gcloud auth application-default login


You are running on a Google Compute Engine virtual machine.
The service credentials associated with this virtual machine
will automatically be used by Application Default
Credentials, so it is not necessary to use this command.

If you decide to proceed anyway, your user credentials may be visible
to others with access to this virtual machine. Are you sure you want
to authenticate with your personal account?

Do you want to continue (Y/n)?  Y

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=AUtl3HTBf2pjpT5CuwkODomGgHetrP&prompt=consent&token_

### **Librerias**

In [1]:
import random
import base64
from collections import Counter
import pandas as pd
import numpy as np
from imblearn.over_sampling import SMOTE, RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score
from sklearn.utils.class_weight import compute_class_weight
import spacy
import nltk
from nltk.tag import pos_tag
import tensorflow as tf
from transformers import pipeline, BertTokenizer, TFBertForSequenceClassification
import torch
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.metrics import Recall
import fasttext
from google.cloud import storage
import vertexai
from vertexai.generative_models import GenerativeModel, Part, SafetySetting

In [2]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_seq_items', None)

### **Funciones**

In [3]:
def list_blobs_prefix(bucket_name, prefix):
  """
  Funcion que obtiene los path en un folder de un bucket de cloud storage.
  """
  list_tmp = []

  storage_client = storage.Client()

  blobs = storage_client.list_blobs(bucket_name, prefix=prefix)

  for blob in blobs:
    list_tmp.append(f"gs://{bucket_name}/{blob.name}")

  return list_tmp

In [4]:
def generate():
  """
  Funcion que genera un caption de un video.
  """

  caption_tmp = ""

  vertexai.init(project='project-team-13', location='us-central1')
  model = GenerativeModel(
      "gemini-1.5-flash-002",
  )
  responses = model.generate_content(
      [video1, text1],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=True,
  )

  for response in responses:
    caption_tmp += response.text

  return caption_tmp

In [5]:
def split_caption(text):
  """
  Funcion que separa el caption de acuerdo a los caracteres '**'.
  """
  split_items= text.split('**')
  if len(split_items) == 5:
    return [split_items[2], split_items[4]]
  else:
    return [None] * 2

In [6]:
def add_pos_tags(text):
  """
  Funcion que agrega pos tags para agregar mas contexto a las palabras.
  """
  tagged_words = pos_tag(nltk.word_tokenize(text))
  return ' '.join([word + '_' + tag for word, tag in tagged_words])

In [7]:
def extract_features(text):
  """
  Funcion que obtiene las features previo a aplicar una clsificacion.
  """
  inputs = tokenizerZS(text, return_tensors='tf')
  outputs = modelZS(**inputs)
  features = outputs.logits
  return features.numpy().flatten()

In [8]:
def classify_text(text, candidate_collision_labels_tmp, candidate_no_collision_labels_tmp):
  """
  Funcion que clasifica un texto a partir de un zero-shot classifier.
  """
  result_1 = classifier(text, candidate_collision_labels_tmp)
  result_2 = classifier(text, candidate_no_collision_labels_tmp)
  if result_1['scores'][0] > result_2['scores'][0]:
    return 1
  else:
    return 0

## **Generacion de captions con Gemini para una muestra de videos**

En esta seccion se generan nuevas transcripciones con Gemini en Vertex de GCP. Se selecciona una muestra aleatoria de 150 videos.

In [ ]:
list_video_train_paths = list_blobs_prefix('bucket-video-wts', 'train/')

In [ ]:
list_path_train = [path for path in list_video_train_paths if 'normal_trimmed' not in path]

In [ ]:
list_video_val_paths = list_blobs_prefix('bucket-video-wts', 'val/')

In [ ]:
list_path_val = [path for path in list_video_val_paths if 'normal_trimmed' not in path]

In [ ]:
list_path = list_path_train + list_path_val

In [ ]:
list_sample_path = random.sample(list_path[1:], 150)

In [ ]:
list_video_captions = []

for gcs_path in list_sample_path:

  video1 = Part.from_uri(
      mime_type="video/mp4",
      uri=gcs_path
  )

  text1 = """Look through each frame in the video carefully and answer the question.
    Only base your answers strictly on what information is available in the video attached.
    Do not make up any information that is not part of the video and do not be too verbose.

    Questions:
    - Give me a detail description of the video, related to the safety of pedestrians with respect to vehicles.
    - In the video, is there a collision or accident (contact or hit between pedestrians and vehicles) for pedestrians with respect to vehicles?"""

  generation_config = {
      "max_output_tokens": 8192,
      "temperature": 1,
      "top_p": 0.95
  }

  safety_settings = [
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HATE_SPEECH,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      ),
      SafetySetting(
          category=SafetySetting.HarmCategory.HARM_CATEGORY_HARASSMENT,
          threshold=SafetySetting.HarmBlockThreshold.OFF
      )
  ]

  list_video_captions.append(generate())

In [ ]:
df_gemini_captions = pd.DataFrame(data=zip(list_sample_path, list_video_captions), columns=['path', 'caption'])

## **Analisis de los datos**

En esta seccion se genera un pandas dataframe con el que se probaran algunos modelos posteriormente.

Se da una primer visualizacion de los datos.

In [ ]:
df_gemini_captions = pd.read_csv('gemini_video_captions3.csv')

In [ ]:
df_gemini_captions.head()

,path,caption
0,gs://bucket-video-wts/train/20230929_68_SN32_T1/vehicle_view/20230929_68_SN32_T1_vehicle_view.mp4,"Here's a description of the video based on your questions:\n\nThe video shows a car approaching a crosswalk at night. Several pedestrians are crossing the road. One pedestrian appears to briefly stop in front of the car, then continue walking. The car slows down but does not stop completely. The road is relatively dark with streetlights and there is a traffic signal visible.\n\n\nThere is no collision or accident between the pedestrians and the vehicle in the video."
1,gs://bucket-video-wts/train/20231006_34_CN24_T1/overhead_view/20231006_34_CN24_T1_192.168.0.13_4.mp4,"Here are the answers based on the provided video:\n\n**Description of the video related to pedestrian safety:**\n\nThe video shows a pedestrian crossing at an intersection with pedestrian crosswalks clearly marked. Two pedestrians are crossing the road while a bicycle is parked near the crosswalk. No vehicles are present during the video.\n\n**Collision or accident?**\n\nNo, there is no collision or accident between pedestrians and vehicles in this video.\n"
2,gs://bucket-video-wts/train/20230707_25_SY1_T1/vehicle_view/20230707_25_SY1_T1_vehicle_view.mp4,"Here are the answers based on the provided video:\n\n**Description of pedestrian safety:** The video shows a car approaching a pedestrian crossing. Pedestrians are crossing the street while the traffic light is green for vehicles. A person is seen bending over near the road as the vehicle passes, with a person holding an umbrella nearby. The vehicles stop before the crosswalk for the pedestrians to cross.\n\n**Collision/Accident:** No, there is no collision or accident between pedestrians and vehicles in the video."
3,gs://bucket-video-wts/val/20230922_19_CN13_T1/overhead_view/20230922_19_CN13_T1_192.168.0.12_3.mp4,"Here are the answers based on the provided video:\n\n- **Description:** The video shows a high-angle, wide view of a driving test area or similar location with various road markings. Pedestrians are crossing a crosswalk while a white car is slowly approaching and stopping to allow the pedestrians to cross safely. Another pedestrian is walking near the roadway.\n\n- **Collision/Accident:** No. There is no collision or accident between pedestrians and the vehicle shown in the video. The car stops for the pedestrians, and they cross without incident."
4,gs://bucket-video-wts/val/20230922_32_CN3_T1/overhead_view/20230922_32_CN3_T1_192.168.0.12_4.mp4,"Here are the answers based on the provided video:\n\n**Description of the video related to pedestrian safety:** The video shows a driving school or similar area with marked pedestrian crossings. A car approaches and stops at the crosswalk while pedestrians cross. The pedestrians and driver appear to follow traffic rules, demonstrating safe interaction between pedestrians and vehicles.\n\n\n**Collision/Accident:** No, there is no collision or accident between pedestrians and vehicles in the video.\n"


Se eliminan los saltos de renglon '\n'.

In [ ]:
df_gemini_captions['caption'] = df_gemini_captions['caption'].str.replace('\n', '', regex=True)

Como se observa las preguntas hechas a Gemini en general quedan delimitadas por dos asteriscos '**'. Por lo que dividimos cada texto con respecto a esos caracteres.

In [ ]:
list_captions = []

for cap in df_gemini_captions['caption']:
  list_captions.append(cap.split('**'))

En principio se esperarian 5 elementos por lista. El primero introduciendo la descripcion, el segundo el titulo de la descripcion, el tercero la descripcion, el cuarto el titulo del evento 'accidente/colision' y quinto la afirmacion/negacion de ese evento.

In [ ]:
dict_count = {}

for l in list_captions:
  lenght = len(l)
  if lenght in dict_count:
    dict_count[lenght] += 1
  else:
    dict_count[lenght] = 1

De los 150 textos disponibles, 134 cumplen con el comportamiento esperado por lo que se separan ambos textos dos nuevas columnas.

In [ ]:
dict_count

{1: 14, 5: 134, 9: 1, 7: 1}

In [ ]:
for l in list_captions:
  if len(l) != 5:
    print(l)

["Here's a description of the video based on your questions:The video shows a car approaching a crosswalk at night. Several pedestrians are crossing the road. One pedestrian appears to briefly stop in front of the car, then continue walking. The car slows down but does not stop completely.  The road is relatively dark with streetlights and there is a traffic signal visible.There is no collision or accident between the pedestrians and the vehicle in the video."]
["Here's a description of the video concerning pedestrian safety and whether a collision occurred:The video shows a car driving at night on a road with minimal traffic. Pedestrians are seen crossing the road in front of the vehicle; however, the driver slows down and safely passes them without incident.  A person briefly appears to be sitting on the car's hood.There is no collision or accident between pedestrians and vehicles shown in the video."]
["Here's a description of the video regarding pedestrian safety and whether a coll

Se valida que las listas con 5 elementos tengan el comportamiento esperado.

In [ ]:
for l in list_captions:
  if len(l) == 5:
    print(l[1], ' - ', l[3])

Description of the video related to pedestrian safety:  -  Collision or accident?
Description of pedestrian safety:  -  Collision/Accident:
Description:  -  Collision/Accident:
Description of the video related to pedestrian safety:  -  Collision/Accident:
Description of the video related to pedestrian safety:  -  Collision or accident:
Description of pedestrian safety:  -  Collision/accident:
Description of the video related to pedestrian safety:  -  Collision or accident?:
Description:  -  Collision/Accident:
Description of pedestrian safety:  -  Collision/accident:
Description:  -  Collision/Accident:
Description of pedestrian safety:  -  Collision/accident:
Description of the video related to pedestrian safety:  -  Collision/Accident:
Description of pedestrian safety:  -  Collision/accident:
Description of the video related to pedestrian safety:  -  Collision or accident?:
Description of pedestrian safety:  -  Collision/Accident:
Description:  -  Collision/Accident:
Description of p

Se crean dos nuevas columnas.

In [ ]:
df_gemini_captions[['description', 'collision/accident']] = df_gemini_captions['caption'].apply(lambda x: pd.Series(split_caption(x)))

Validamos los valores que quedaron en 'None'.

In [ ]:
df_gemini_captions[df_gemini_captions['description'].isnull()]

,path,caption,description,collision/accident
0,gs://bucket-video-wts/train/20230929_68_SN32_T1/vehicle_view/20230929_68_SN32_T1_vehicle_view.mp4,"Here's a description of the video based on your questions:The video shows a car approaching a crosswalk at night. Several pedestrians are crossing the road. One pedestrian appears to briefly stop in front of the car, then continue walking. The car slows down but does not stop completely. The road is relatively dark with streetlights and there is a traffic signal visible.There is no collision or accident between the pedestrians and the vehicle in the video.",None,None
6,gs://bucket-video-wts/val/20230728_60_SY11_T1/vehicle_view/20230728_60_SY11_T1_vehicle_view.mp4,"Here's a description of the video concerning pedestrian safety and whether a collision occurred:The video shows a car driving at night on a road with minimal traffic. Pedestrians are seen crossing the road in front of the vehicle; however, the driver slows down and safely passes them without incident. A person briefly appears to be sitting on the car's hood.There is no collision or accident between pedestrians and vehicles shown in the video.",None,None
11,gs://bucket-video-wts/train/20230728_31_CN37_T1/overhead_view/20230728_31_CN37_T1_Camera3_1.mp4,Here's a description of the video regarding pedestrian safety and whether a collision occurred:The video shows a driving test area. Pedestrians are crossing a marked crosswalk. A car approaches the crosswalk and stops to allow the pedestrians to cross. The car then proceeds after pedestrians have crossed.No collision or accident occurs between pedestrians and the vehicle.,None,None
23,gs://bucket-video-wts/train/20230922_38_CN7_T1/overhead_view/20230922_38_CN7_T1_192.168.0.11_1.mp4,"Here are the answers based on the provided video:- The video shows a pedestrian crossing at a traffic intersection with pedestrian crosswalks and traffic signals. A white car approaches the crosswalk while pedestrians are crossing, but the car stops to allow them to proceed safely. The traffic light is red for the car.- No, there is no collision or accident between pedestrians and vehicles in the video.",None,None
24,gs://bucket-video-wts/train/20230922_47_SN37_T1/overhead_view/20230922_47_SN37_T1_192.168.0.11_1.mp4,"Here are the answers based on the provided video:- The video shows a pedestrian crossing with traffic signals. Pedestrians are seen crossing only when the traffic signal is green for pedestrians (and red for vehicles). There are clearly marked pedestrian crossings.- No, there is no collision or accident between pedestrians and vehicles in the video.",None,None
41,gs://bucket-video-wts/val/20230728_27_CN34_T1/overhead_view/20230728_27_CN34_T1_Camera3_1.mp4,"Here are the answers based on the provided video:- The video shows a driving test course. Pedestrians are crossing a marked crosswalk. A car approaches and stops to allow pedestrians to cross safely. The car then proceeds after the pedestrians have completely crossed.- No, there is no collision or accident between pedestrians and vehicles in the video.",None,None
68,gs://bucket-video-wts/train/20231006_25_SN20_T1/overhead_view/20231006_25_SN20_T1_192.168.0.12_2.mp4,"Here's a description of the video concerning pedestrian safety and whether a collision occurs:The video shows a person on a skateboard crossing a road in a driving school area. A car is approaching the pedestrian, who stops before reaching the vehicle. The car stops, and the pedestrian appears to talk to the driver before continuing across the road.No collision or accident occurs between the pedestrian and the vehicle.",None,None
72,gs://bucket-video-wts/train/20230707_19_CN8_T1/overhead_view/20230707_19_CN8_T1_Camera1_0.mp4,"Here are the answers based on the provided video:- The video shows a relatively empty intersection with pedestrians crossing a crosswalk. A car approaches the intersection and stops before the crosswalk while pedestrians are crossing. Pedestrians cros

Reemplazamos los valores.

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[0], 'description'] = "The video shows a car approaching a crosswalk at night. Several pedestrians are crossing the road. One pedestrian appears to briefly stop in front of the car, then continue walking. The car slows down but does not stop completely. The road is relatively dark with streetlights and there is a traffic signal visible."
df_gemini_captions.loc[df_gemini_captions.index[0], 'collision/accident'] = "There is no collision or accident between the pedestrians and the vehicle in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[6], 'description'] = "The video shows a car driving at night on a road with minimal traffic. Pedestrians are seen crossing the road in front of the vehicle; however, the driver slows down and safely passes them without incident. A person briefly appears to be sitting on the car's hood."
df_gemini_captions.loc[df_gemini_captions.index[6], 'collision/accident'] = "There is no collision or accident between pedestrians and vehicles shown in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[11], 'description'] = "The video shows a driving test area. Pedestrians are crossing a marked crosswalk. A car approaches the crosswalk and stops to allow the pedestrians to cross. The car then proceeds after pedestrians have crossed."
df_gemini_captions.loc[df_gemini_captions.index[11], 'collision/accident'] = "No collision or accident occurs between pedestrians and the vehicle."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[23], 'description'] = "The video shows a pedestrian crossing at a traffic intersection with pedestrian crosswalks and traffic signals. A white car approaches the crosswalk while pedestrians are crossing, but the car stops to allow them to proceed safely. The traffic light is red for the car."
df_gemini_captions.loc[df_gemini_captions.index[23], 'collision/accident'] = "No, there is no collision or accident between pedestrians and vehicles in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[24], 'description'] = "The video shows a pedestrian crossing with traffic signals. Pedestrians are seen crossing only when the traffic signal is green for pedestrians (and red for vehicles). There are clearly marked pedestrian crossings."
df_gemini_captions.loc[df_gemini_captions.index[24], 'collision/accident'] = "No, there is no collision or accident between pedestrians and vehicles in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[41], 'description'] = "The video shows a driving test course. Pedestrians are crossing a marked crosswalk. A car approaches and stops to allow pedestrians to cross safely. The car then proceeds after the pedestrians have completely crossed."
df_gemini_captions.loc[df_gemini_captions.index[41], 'collision/accident'] = "No, there is no collision or accident between pedestrians and vehicles in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[68], 'description'] = "The video shows a person on a skateboard crossing a road in a driving school area. A car is approaching the pedestrian, who stops before reaching the vehicle. The car stops, and the pedestrian appears to talk to the driver before continuing across the road."
df_gemini_captions.loc[df_gemini_captions.index[68], 'collision/accident'] = "No collision or accident occurs between the pedestrian and the vehicle."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[72], 'description'] = "The video shows a relatively empty intersection with pedestrians crossing a crosswalk. A car approaches the intersection and stops before the crosswalk while pedestrians are crossing. Pedestrians cross the crosswalk and the car waits until they have completed their crossing before proceeding through the intersection. The traffic light is red for vehicles while pedestrians are crossing."
df_gemini_captions.loc[df_gemini_captions.index[72], 'collision/accident'] = "No, there is no collision or accident between pedestrians and vehicles in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[73], 'description'] = "The video shows a scene at what appears to be a driving school or test area. A white car stops to allow pedestrians to cross the road. Pedestrians are then shown walking toward the car and passing it without incident. One person appears to be using a cell phone while waiting to cross."
df_gemini_captions.loc[df_gemini_captions.index[73], 'collision/accident'] = "There is no collision between any pedestrian and any vehicle."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[95], 'description'] = "The video shows a street intersection with pedestrians and cyclists crossing. There is a traffic light present, but the level of adherence to the traffic signals by pedestrians and cyclists is not fully clear. The video is shot from a static point of view and does not show the street from all angles."
df_gemini_captions.loc[df_gemini_captions.index[95], 'collision/accident'] = "There is no collision or accident shown in the video between pedestrians or cyclists and vehicles. However, there is an incident involving a cyclist falling near the crosswalk."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[98], 'description'] = "The video shows a pedestrian crossing at a crosswalk. A cyclist also crosses the road at the crosswalk. Traffic signals are present to regulate the traffic and pedestrian flow. The video shows the traffic light turning red allowing pedestrians to cross safely."
df_gemini_captions.loc[df_gemini_captions.index[98], 'collision/accident'] = "No, there is no collision or accident shown in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[103], 'description'] = "The video shows a car approaching a pedestrian crossing. Two pedestrians are waiting at the crossing, and the traffic light is red for the car. The car stops, allowing the pedestrians to cross safely."
df_gemini_captions.loc[df_gemini_captions.index[103], 'collision/accident'] = "No, there is no collision or accident between pedestrians and vehicles in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[104], 'description'] = "The video shows a wide shot of a multi-lane intersection with pedestrian crosswalks. Several pedestrians and a cyclist are using the crosswalks. A white car approaches the intersection while the pedestrians are crossing. The car slows down and safely passes the pedestrians and cyclist in the crosswalk. The traffic light is visible, and it appears to change from red to yellow during the video."
df_gemini_captions.loc[df_gemini_captions.index[104], 'collision/accident'] = "No, there is no collision or accident (no contact or hit between pedestrians and vehicles) in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[108], 'description'] = "The video shows a driving test area. Several people are practicing driving maneuvers in cars and on bicycles. Pedestrians are also present in the area. Some pedestrians walk near the vehicles, and at one point, a cyclist nearly collides with a car. The video shows several instances where pedestrian safety is not prioritized. Pedestrians are walking near moving vehicles without clear separation or right-of-way. A cyclist nearly collides with a car because he has no safe zone to use."
df_gemini_captions.loc[df_gemini_captions.index[108], 'collision/accident'] = "No direct collision or accident between pedestrians and vehicles occurs in the video. However, a near miss happens between a cyclist and a car."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[121], 'description'] = "The video shows a car's rear view, looking out at a seemingly empty driving range or similar area. There are no pedestrians visible in the driving range area, and no indication of pedestrian crossings or walkways. There is a fence separating the driving range and the road, suggesting a measure to protect pedestrians."
df_gemini_captions.loc[df_gemini_captions.index[121], 'collision/accident'] = " No, there is no collision or accident between pedestrians and vehicles shown in the video."

In [ ]:
df_gemini_captions.loc[df_gemini_captions.index[134], 'description'] = "The video shows a high-angle view of a road intersection with pedestrian crosswalks. Several pedestrians are crossing the road at different times. A white car approaches the intersection and stops to allow pedestrians to cross before continuing. The video shows a scenario where drivers yield to pedestrians in a crosswalk, and pedestrians appear to be aware of oncoming traffic."
df_gemini_captions.loc[df_gemini_captions.index[134], 'collision/accident'] = "No, there is no collision or accident between pedestrians and vehicles in the video."

In [ ]:
df_gemini_captions['class'] = df_gemini_captions['collision/accident'].apply(lambda x: 1 if 'yes' in x.lower() else 0)

### Datos de entrenamiento y prueba

In [9]:
df_gemini_captions = pd.read_csv('video_captions.csv')

In [10]:
X_train, X_test, y_train, y_test = train_test_split(df_gemini_captions['caption'], df_gemini_captions['class'], test_size=0.2, random_state=7)

## **Ingenieria de caracteristicas y modelos**

### **Regresion Logistica**

### Ingenieria de caracteristicas con POS TAGS

In [11]:
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [12]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
X_train_tag = X_train.apply(add_pos_tags)
X_test_tag = X_test.apply(add_pos_tags)

In [14]:
vectorizer = TfidfVectorizer()

In [15]:
X_train_vec = vectorizer.fit_transform(X_train_tag)
X_test_vec = vectorizer.transform(X_test_tag)

In [16]:
ros = RandomOverSampler(random_state=7)

In [17]:
X_train_vec_resampled, y_train_resampled = ros.fit_resample(X_train_vec, y_train)

### Modelo con POS TAGS

In [18]:
model = LogisticRegression()
model.fit(X_train_vec_resampled, y_train_resampled)

LogisticRegression()

In [19]:
y_pred = model.predict(X_test_vec)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

In [21]:
print('Accuracy: ', accuracy)

Accuracy:  0.9333333333333333


In [22]:
print('Recall: ', recall)

Recall:  0.0


### Ingenieria de caracteristicas con Features

In [23]:
tokenizerZS = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [24]:
modelZS = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
df_gemini_captions['features'] = df_gemini_captions['caption'].apply(extract_features)

In [28]:
df_gemini_captions_train = df_gemini_captions.sample(frac=0.8, random_state=7)
df_gemini_captions_test = df_gemini_captions.drop(df_gemini_captions_train.index)

In [29]:
X_train_features = df_gemini_captions_train['features'].tolist()
y_train_features = df_gemini_captions_train['class'].tolist()

### Modelo con Features

In [30]:
classifier = LogisticRegression()

In [32]:
classifier.fit(X_train_features, y_train_features)

LogisticRegression()

In [33]:
X_test_features = df_gemini_captions_test['features'].tolist()
y_test_features = df_gemini_captions_test['class'].tolist()

In [34]:
y_pred_features = classifier.predict(X_test_features)

In [38]:
accuracy = accuracy_score(y_test_features, y_pred_features)
recall = recall_score(y_test_features, y_pred_features, zero_division=0.0)

In [39]:
print('Accuracy: ', accuracy)

Accuracy:  1.0


In [40]:
print('Recall: ', recall)

Recall:  0.0


### **Red Neuronal Analisis de Sentimientos**

### Ingeniria de caracteristicas

In [41]:
tokenizer = Tokenizer(num_words=10000)

In [42]:
tokenizer.fit_on_texts(X_train)

In [43]:
X_train_token = tokenizer.texts_to_sequences(X_train)

In [44]:
X_test_token = tokenizer.texts_to_sequences(X_test)

In [45]:
max_length = 256

In [46]:
X_train_token = pad_sequences(X_train_token, maxlen = max_length)

In [47]:
X_test_token = pad_sequences(X_test_token, maxlen = max_length)

### Modelo

In [48]:
model = Sequential()
model.add(Embedding(10000, 128))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

In [49]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Recall()])

In [50]:
ros = RandomOverSampler(random_state=7)

In [51]:
X_train_token_resampled, y_train_resampled = ros.fit_resample(X_train_token, y_train)

In [52]:
model.fit(X_train_token_resampled, y_train_resampled, epochs=1, batch_size=64, validation_split=0.2)

3/3 ━━━━━━━━━━━━━━━━━━━━ 3s 442ms/step - accuracy: 0.4914 - loss: 0.6797 - recall: 0.4436 - val_accuracy: 1.0000 - val_loss: 0.6816 - val_recall: 1.0000


In [53]:
_, accuracy, recall = model.evaluate(X_test_token, y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step - accuracy: 0.9333 - loss: 0.5862 - recall: 0.0000e+00


In [54]:
print('Accuracy: ', accuracy)

Accuracy:  0.9333333373069763


In [55]:
print('Recall: ', recall)

Recall:  0.0


### **fastText**

### Modelo

In [56]:
with open('train.txt', 'w') as f:
  for text, label in zip(X_train, y_train):
    f.write(f"__label__{label} {text}\n")

with open('test.txt', 'w') as f:
  for text, label in zip(X_test, y_test):
    f.write(f"__label__{label} {text}\n")

In [57]:
model = fasttext.train_supervised(input='train.txt', epoch=25, lr=1.0, wordNgrams=2, verbose=2, minCount=1)

In [58]:
test_result = model.test('test.txt')

In [59]:
print(test_result[1])

0.9333333333333333


## **Modelo final**

Como modelo final se eligio una clasificacion con Zero-Shot, tuvo un performance aceptable con las metricas elegidas: Accuracy y Recall. Aunque el presente Notebook lo consideramos un primer paso de una analisis que se deberia llevar de manera mas profunda en el futuro.

### **Zero-shot classification**

In [61]:
if torch.cuda.is_available():
    classifier = pipeline('zero-shot-classification', model = 'facebook/bart-large-mnli', device="cuda")
else:
    classifier = pipeline('zero-shot-classification', model = 'facebook/bart-large-mnli')

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [62]:
candidate_collision_labels = ['a collision', 'accident', 'hitting']
candidate_no_collision_labels = ['no collision', 'no accident']

In [65]:
df_gemini_captions['prediction'] = df_gemini_captions.apply(lambda x: classify_text(x['caption'], candidate_collision_labels, candidate_no_collision_labels), axis=1)

In [68]:
accuracy = accuracy_score(df_gemini_captions['class'], df_gemini_captions['prediction'])

In [67]:
recall = recall_score(df_gemini_captions['class'], df_gemini_captions['prediction'])

In [69]:
print('Accuracy: ', accuracy)

Accuracy:  0.9866666666666667


In [70]:
print('Recall: ', recall)

Recall:  0.5


## **Conclusiones**

En el presente analisis se utilizaron como datos las transcripciones generadas con Gemini de 150 videos de la base WTS, en las que se le solicito a Gemini dar una descripcion del video asi como una clasificacion de si habia alguna colision o accidente, la clasificacion de Gemini se considero como dada ante la ejecucion de los modelos seleccionados. Cabe señalar que de los 150 videos Gemini solo clasifico 4 en los que habia una colision o accidente, los cuales fueron validados. Se probaron los modelos: Regresion Logistica, Red Neuronal, fastText y Zero-Shot. En los dos primeros casos se busco aplicar algunas tecnicas de feature engineering como tokenizaciones y POS TAG. Dado que se tenia una clase desbalanceada el Zero-Shot presento el mejor desempeño. Consideramos que el prsente analisis es un primer paso para seguir explorando los conflictos viales.